#Configuration Parameters

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
import os
import argparse
import csv

import imageio
import numpy as np
import pandas as pd

from keras.utils import multi_gpu_model, to_categorical
from keras.layers import Activation,Input
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow import convert_to_tensor
from tensorflow.keras.metrics import MeanIoU
from keras.models import load_model

sys.path.append("/content/drive/My Drive/tf-keras-SegNet")
from model import segnet
from layers import MaxPoolingWithArgmax2D, MaxUnpooling2D
sys.path.append('/content/drive/My Drive')
from lasero.utils import training

Using TensorFlow backend.


In [0]:
base_dir = "/content/drive/My Drive/"

In [0]:
def get_image_list(metadata_file_path):
    with open(metadata_file_path, 'r') as f:
        return [i for i in csv.reader(f) if i]

In [5]:
dir_indices = 0, 0, 1
model_dir = os.path.join(base_dir, 'models')
input_components = [i for i in os.listdir(model_dir) if 'dataset' in i]
print(input_components)
selected_input_component = input_components[dir_indices[0]]
print(selected_input_component)
input_component_dir = os.path.join(model_dir, selected_input_component)
print()

opt_components = [i for i in os.listdir(input_component_dir)]
print(opt_components)
selected_opt_component = opt_components[dir_indices[1]]
print(selected_opt_component)
opt_component_dir = os.path.join(input_component_dir, selected_opt_component)
print()

callback_components = [i for i in os.listdir(opt_component_dir)]
print(callback_components)
selected_callback_component = callback_components[dir_indices[2]]
print(selected_callback_component)
print()

session_name = os.path.join(selected_input_component, selected_opt_component, selected_callback_component)

source_session_paths = training.create_session_paths(session_name)
for i in source_session_paths:
    print(source_session_paths[i])

print('\navailable models:')

for i in os.listdir(source_session_paths['session_dir']):
    print(i)

['dataset=one_percent_bands=1-4-8-_weights=none', 'dataset=one_percent_bands=1-2-3-_weights=none', 'dataset=one_percent_bands=0-1-2-3-4-5-6-7-8-9-_weights=none']
dataset=one_percent_bands=1-4-8-_weights=none

['opt=Adam_loss=categorical_crossentropy_lr=0.001']
opt=Adam_loss=categorical_crossentropy_lr=0.001

['batch=4_epochs=50_lrmonitor=val_loss_factor=0.5_patience=3_min=0.0001_stopmonitor=val_loss_patience=9', 'batch=5_epochs=50_lrmonitor=val_loss_factor=0.5_patience=3_min=0.0001_stopmonitor=val_loss_patience=9', 'batch=5_epochs=10_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9', 'batch=5_epochs=10_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20']
batch=5_epochs=50_lrmonitor=val_loss_factor=0.5_patience=3_min=0.0001_stopmonitor=val_loss_patience=9

/content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=

In [0]:
session_manager = training.SessionManager(base_dir)
input_setting = session_manager.parse_component(selected_input_component)
opt_setting = session_manager.parse_component(selected_opt_component)
callback_setting = session_manager.parse_component(selected_callback_component)

In [7]:
print(input_setting)
print(opt_setting)
print(callback_setting)

{'dataset': 'one_percent', 'bands': ['1', '4', '8'], 'weights': 'none'}
{'opt': 'Adam', 'loss': 'categorical_crossentropy', 'lr': '0.001'}
{'batch': '5', 'epochs': '50', 'lrmonitor': 'val_loss', 'factor': '0.5', 'patience': '9', 'min': '0.0001', 'stopmonitor': 'val_loss'}


In [0]:
metadata_path = os.path.join(base_dir, "Metadata")
dataset = input_setting['dataset']
training_set_path = os.path.join(metadata_path, f'{dataset}_train.csv')
val_set_path = os.path.join(metadata_path, f'{dataset}_val.csv')

bands = [int(i) for i in input_setting['bands']] # number of bands

# if 'none', transfer weights cells should be skipped
weights = 'none'

num_classes = 2
class_weight = {0:1,1:1}

loss_function = opt_setting['loss']
learning_rate = float(opt_setting['lr'])
new_lr = 0.0005
optimizer = Adam(learning_rate=new_lr)

batch_size = int(callback_setting['batch'])
epochs = 50
iou = MeanIoU(num_classes)
metrics = ['accuracy', iou]
source_model_name = 'epoch20'

In [0]:
lr_monitor_metric = 'val_loss'
factor = 0.5
lr_patience = 2
min_lr = 0.00005

reduce_lr = ReduceLROnPlateau(monitor=lr_monitor_metric,
                              factor=factor,
                              patience=lr_patience,
                              verbose=1,
                              mode='auto',
                              min_lr=min_lr)

stop_monitor_metric = 'val_loss'
stop_patience = 9

early_stopper = EarlyStopping(monitor=stop_monitor_metric,
                              patience=stop_patience,
                              verbose=1,
                              mode='auto')

In [44]:
session_manager = training.SessionManager(base_dir)
input_component = session_manager.build_input_components(dataset, bands, weights)

opt_component = session_manager.build_opt_components(optimizer.__class__.__name__, loss_function, learning_rate)

training_components = session_manager.build_training_components(batch_size, epochs, class_weight)
lr_monitor_components = session_manager.build_lr_monitor_components(lr_monitor_metric, factor, lr_patience, min_lr)
stop_monitor_components = session_manager.build_stop_components(stop_monitor_metric, stop_patience)

model_prefix = training_components + lr_monitor_components + stop_monitor_components

session_path = os.path.join(input_component, opt_component, model_prefix + f'_source={source_model_name}')
print(session_path)

dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20


In [45]:
session_paths = training.create_session_paths(session_path)
for i in session_paths:
    print(session_paths[i])

/content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20
/content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/history.json
/content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/logs.csv
/content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min

In [0]:
training_set_list = training.get_image_list(training_set_path)
val_set_list = training.get_image_list(val_set_path)
training_set_size = len(training_set_list)
val_set_size = len(val_set_list)

image_shape = (512, 512, len(bands))

# Transfer weights of matching layers from image-net trained vgg16 to segnet

In [47]:
segnet_model = segnet(image_shape, num_classes)
if weights != 'none':
    vgg_model = VGG16(include_top=False, weights=weights, input_shape=image_shape, classes=num_classes)
    seg_layer_names = [i for i in segnet_model.layers if "conv" in i.name]
    vgg_layer_names = [i for i in vgg_model.layers if "conv" in i.name]
    
    transferable_layer_names = {}
    for i in range(len(vgg_layer_names)):
        transferable_layer_names[seg_layer_names[i].name] = vgg_layer_names[i]
    
    layer_count = 0
    for i in segnet_model.layers:
        try:
            i.set_weights(transferable_layer_names[i.name].get_weights())
            layer_count += 1
        except KeyError:
            pass
    
    print(layer_count)

Build enceder done..
Build decoder done..


# Custom Generator

In [0]:
def data_gen(metadata_file_path, bands, batch_size):
    image_list = np.asarray(get_image_list(metadata_file_path))
    np.random.seed(1)
    np.random.shuffle(image_list)

    band_normalization_map = []
    counter = 0

    total_steps = image_list.shape[0] // batch_size
    while True:
        step_start = counter * batch_size
        step_end = step_start + batch_size
        images = []
        masks = []
        for j in range(step_start, step_end):
            images.append(np.load(image_list[j, 0])[:,:,bands])
            masks.append(np.load(image_list[j, 1]))

        y = to_categorical(np.array(masks), 2)
        yield np.array(images) / 65535, y.reshape((batch_size, y.shape[1] * y.shape[2], y.shape[3])) 

        counter +=1

        if counter >= total_steps:
            counter = 0
            np.random.shuffle(image_list)

In [0]:
train_df = pd.read_csv(training_set_path)

#Training model 


In [0]:
segnet_model = load_model(os.path.join(source_session_paths['session_dir'], f'{source_model_name}.h5'), custom_objects={'MaxPoolingWithArgmax2D': MaxPoolingWithArgmax2D, 'MaxUnpooling2D': MaxUnpooling2D}, compile=False)

In [0]:
train_data = data_gen(training_set_path, bands, batch_size)
val_data = data_gen(val_set_path, bands, batch_size)

segnet_model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

accuracy_checkpoint = ModelCheckpoint(os.path.join(session_paths['session_dir'], 'val_accuracy.h5'),
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')

loss_checkpoint = ModelCheckpoint(os.path.join(session_paths['session_dir'], 'val_loss.h5'),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='auto')

iou_checkpoint = ModelCheckpoint(os.path.join(session_paths['session_dir'], 'MeanIoU.h5'),
                             monitor='val_' + segnet_model.metrics[1].name,
                             verbose=1,
                             save_best_only=True,
                             mode='max')

# concatenation of save path is needed here to save model name dynamically by epoch
checkpoint = ModelCheckpoint(session_paths['session_dir'] + "/epoch{epoch:02d}.h5",
                             period=2,
                             save_weights_only=False,
                             save_best_only=False)

csv_logger = CSVLogger(session_paths["logs"])

callbacks_list = [iou_checkpoint, accuracy_checkpoint, loss_checkpoint, checkpoint, csv_logger]

hist = segnet_model.fit(train_data,
                        steps_per_epoch=training_set_size // batch_size,
                        epochs=epochs,
                        validation_data=val_data,
                        validation_steps=val_set_size // batch_size,
                        verbose=1,
                        callbacks=callbacks_list
                        )

Epoch 1/50
154/154 [==============================] - 224s 1s/step - loss: 0.0952 - accuracy: 0.9691 - mean_io_u_4: 0.2512 - val_loss: 0.0703 - val_accuracy: 0.9386 - val_mean_io_u_4: 0.2512

Epoch 00001: val_mean_io_u_4 improved from -inf to 0.25116, saving model to /content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/MeanIoU.h5

Epoch 00001: val_accuracy improved from -inf to 0.93861, saving model to /content/drive/My Drive/models/dataset=one_percent_bands=1-4-8-_weights=none/opt=Adam_loss=categorical_crossentropy_lr=0.001/batch=5_epochs=50_classweight=1-1-_lrmonitor=val_loss_factor=0.5_patience=2_min=5e-05_stopmonitor=val_loss_patience=9_source=epoch20/val_accuracy.h5

Epoch 00001: val_loss improved from inf to 0.07026, saving model to /content/drive/My Drive/models/dataset=one_per